In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pandas as pd
from random import randint

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
path = '/mnt/car_brand_attribute_associations/'

In [0]:
wd = webdriver.Chrome(options=chrome_options)

data = []
random_numbers = set()

random = 0
if random == 1:
    while len(random_numbers) != 100:
        random_number = randint(1, 436)
        random_numbers.add(random_number)
else:
    for i in range(1, 110):
        random_numbers.add(i)   

for i in (pbar := tqdm(list(random_numbers))):
    if i==1:
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans'
    else:
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans' + "/p" + str(i)
    
    wd.get(url)

    comments = wd.find_elements(By.CLASS_NAME, value="Comment")

    for comment in comments:
        user = comment.find_element(By.TAG_NAME, "a").get_attribute('href')
        user = user.split('/')[-1]
        
        posts = comment.find_element(by=By.CLASS_NAME, value="AuthorInfo")
        posts = posts.find_element(by=By.TAG_NAME, value="b").text
        posts = posts.split(',')
        posts = "".join(posts)
        
        time = comment.find_element(by=By.TAG_NAME, value="time").get_attribute("datetime")
        
        comment_text = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap").text
        comment_text = comment_text.split()
        comment_text = " ".join(comment_text)
        comment_text = comment_text.strip()

        data.append([user, time, comment_text, posts])
    pbar.set_description(f"Collected {len(data)} rows from url: {url}")

data

In [0]:
data = pd.DataFrame(data, columns=['User', 'Post Time', 'Comment', '#Posts by User'])
data['Post Time'] = pd.to_datetime(data['Post Time'])
data = data.astype({'#Posts by User': int})
data = data.loc[~data['Comment'].isnull()]
data = data.loc[~data['Post Time'].isnull()]
data = data[:5000]
df_spark = spark.createDataFrame(data)
df_spark.write.option("header", "true").mode("overwrite").csv(path+'Data/Edmunds_Data_Old')
data.info()

In [0]:
wd = webdriver.Chrome(options=chrome_options)

n = 6000
data = {}
user_dict = {}
posts_dict = {}
time_dict = {}
i = 435
while len(data) < n:
    url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans' + "/p" + str(i)
    wd.get(url)

    comments = wd.find_elements(By.CLASS_NAME, value="Comment")

    for comment in reversed(comments): 
        user = comment.find_element(By.TAG_NAME, "a").get_attribute('href')
        user = user.split('/')[-1]
        
        posts = comment.find_element(by=By.CLASS_NAME, value="AuthorInfo")
        posts = posts.find_element(by=By.TAG_NAME, value="b").text
        posts = posts.split(',')
        posts = "".join(posts)
        
        time = comment.find_element(by=By.TAG_NAME, value="time").get_attribute("datetime")

        try:
            backquote = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
            backquote = backquote.find_element(by=By.TAG_NAME, value="blockquote")
            backquote = backquote.find_element(by=By.CLASS_NAME, value="QuoteText.blockquote-content").text
            backquote = backquote.split()
            backquote = " ".join(backquote)
            backquote = backquote.strip()
        except:
            backquote = ""
        
        comment_text = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
        comment_text = comment.find_element(by=By.CLASS_NAME, value="Message.userContent").text
        comment_text = comment_text.split()
        comment_text = " ".join(comment_text)
        comment_text = comment_text.strip()

        try:
            sub_text = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
            sub_text = sub_text.find_element(by=By.TAG_NAME, value="blockquote").text
            sub_text = sub_text.split()
            sub_text = " ".join(sub_text)
            sub_text = sub_text.strip()
        except:
            sub_text = ''

        comment_text = comment_text.replace(sub_text, "").strip()

        user_dict[comment_text] = user
        posts_dict[comment_text] = posts
        time_dict[comment_text] = time

        if backquote != "":
            if comment_text in data.keys():
                text = [comment_text]
                for j in data[comment_text]:
                    text.append(j)
                data[backquote] = text
                del data[comment_text]
            else:
                data[backquote] = [comment_text]
        else:
            if comment_text not in data.keys():
                data[comment_text] = []

    l = 30
    d = int((len(data)/n) * l)
    sys.stdout.write("\rCollecing samples from {1}: {0}>".format("="*d + "."*(l-d-1), url))
    sys.stdout.flush()
    i = int(i)
    i = i - 1
wd.quit()

In [0]:
edmunds = []
for key in data.keys():
    text = ' '
    text += key
    for j in data[key]:
        text += ' ' + j
    text.strip()

    try:
        user = user_dict[key]
    except KeyError:
        user = None
    
    try: 
        n_posts = posts_dict[key]
    except KeyError:
        n_posts = None

    try:
        time = time_dict[key]
    except KeyError:
        time = None
    
    edmunds.append([user, time, text, n_posts])

edmunds = pd.DataFrame(edmunds, columns=['User', 'Post Time', 'Comment', '#Posts by User'])
edmunds['Post Time'] = pd.to_datetime(edmunds['Post Time'])
edmunds = edmunds.loc[~edmunds['Comment'].isnull()]
edmunds = edmunds.loc[~edmunds['Post Time'].isnull()]
edmunds = edmunds[:5000]
df_spark = spark.createDataFrame(edmunds)
df_spark.write.option("header", "true").mode("overwrite").csv(path+'Data/Edmunds_Data_New')
edmunds.info()